# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [48]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [49]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [50]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [51]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [52]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(["cassandra"])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [53]:
keyspace = 'sparkifydb'

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS %s
    WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
    """ % keyspace)
except Exception as e:
    print("an error occured while creating keyspace!", e)

#### Set Keyspace

In [54]:
try:
    session.set_keyspace(keyspace)
except Exception as e:
    print("an error occured while setting keyspace!", e)    

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [55]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = """
CREATE TABLE songs_per_session (
    session_id bigint,
    item_in_session int,
    artist varchar,
    first_name varchar,
    gender varchar,
    last_name varchar,
    length decimal,
    level varchar,
    location varchar,
    song varchar,
    user_id bigint,
    PRIMARY KEY (session_id, item_in_session)
)
"""

try:
    session.execute("DROP TABLE IF EXISTS songs_per_session")
    session.execute(query)
except Exception as e:
    print("an error occured while creating table!", e)

"""
Description of table:
    `songs_per_session` table enables to query which song was played in specific session
"""

'\nDescription of table:\n    `songs_per_session` table enables to query which song was played in specific session\n'

In [56]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

from decimal import Decimal

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_per_session (session_id, item_in_session, artist, first_name, gender, last_name, length, level, location, song, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], Decimal(line[5]), line[6], line[7], line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [57]:
query = "SELECT artist, song, length FROM songs_per_session WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print("an error occured while querying!", e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [58]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = """
CREATE TABLE songs_per_user_session (
    user_id bigint,
    session_id bigint,
    item_in_session int,
    artist varchar,
    first_name varchar,
    gender varchar,
    last_name varchar,
    length decimal,
    level varchar,
    location varchar,
    song varchar,
    PRIMARY KEY (user_id, session_id, item_in_session)
)
"""

try:
    session.execute("DROP TABLE IF EXISTS songs_per_user_session")
    session.execute(query)
except Exception as e:
    print("an error occured while creating table!", e)

"""
Description of table:
    `songs_per_user_session` table enables to query which song was played by the specific user and session
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_per_user_session (user_id, session_id, item_in_session, artist, first_name, gender, last_name, length, level, location, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], Decimal(line[5]), line[6], line[7], line[9]))

query = "SELECT artist, song, first_name, last_name FROM songs_per_user_session WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print("an error occured while querying!", e)

for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)
        

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [59]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = """
CREATE TABLE users_per_song (
    song varchar,
    user_id bigint,
    artist varchar,
    first_name varchar,
    gender varchar,
    item_in_session int,
    last_name varchar,
    length decimal,
    level varchar,
    location varchar,
    session_id bigint,
    PRIMARY KEY (song, user_id)
)
"""

try:
    session.execute("DROP TABLE IF EXISTS users_per_song")
    session.execute(query)
except Exception as e:
    print("an error occured while creating table!", e)

"""
Description of table:
    `users_per_song` table enables to query which user played the specific song
"""

'\nDescription of table:\n    `users_per_song` table enables to query which user played the specific song\n'

In [60]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_per_song (song, user_id, artist, first_name, gender, item_in_session, last_name, length, level, location, session_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[2], int(line[3]), line[4], Decimal(line[5]), line[6], line[7], int(line[8])))

In [61]:
query = "SELECT first_name, last_name FROM users_per_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print("an error occured while querying!", e)

for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [62]:
## TO-DO: Drop the table before closing out the sessions

In [63]:
try:
    session.execute("DROP TABLE IF EXISTS users_per_song")
except Exception as e:
    print("an error occured while dropping table!", e)

try:
    session.execute("DROP TABLE IF EXISTS songs_per_user")
except Exception as e:
    print("an error occured while dropping table!", e)
    
try:
    session.execute("DROP TABLE IF EXISTS songs_per_session")
except Exception as e:
    print("an error occured while dropping table!", e)

### Close the session and cluster connection¶

In [64]:
session.shutdown()
cluster.shutdown()